In [1]:
import os
import sys
import numpy as np
import pydot
import rmgpy.tools.fluxdiagram
import rmgpy.chemkin
# from rmgpy.solver.liquid import LiquidReactor

import matplotlib.pyplot as plt
%matplotlib inline

sys.path.append('/home/moon/autoscience/reaction_calculator/database')
import database_fun

using default DFT_DIR


## Settings

In [2]:
# Options controlling the individual flux diagram renderings:
program = 'dot'  # The program to use to lay out the nodes and edges
max_node_count = 50  # The maximum number of nodes to show in the diagram
max_edge_count = 50  # The maximum number of edges to show in the diagram
concentration_tol = 1e-6  # The lowest fractional concentration to show (values below this will appear as zero)
species_rate_tol = 1e-6  # The lowest fractional species rate to show (values below this will appear as zero)
max_node_pen_width = 10.0  # The thickness of the border around a node at maximum concentration
max_edge_pen_width = 10.0  # The thickness of the edge at maximum species rate
radius = 1  # The graph radius to plot around a central species
central_reaction_count = None  # The maximum number of reactions to draw from each central species (None draws all)
# If radius > 1, then this is the number of reactions from every species

# Options controlling the ODE simulations:
initial_time = 1e-12  # The time at which to initiate the simulation, in seconds
time_step = 10 ** 0.1  # The multiplicative factor to use between consecutive time points
abs_tol = 1e-16  # The absolute tolerance to use in the ODE simluations
rel_tol = 1e-8  # The relative tolerance to use in the ODE simulations

# Options controlling the generated movie:
video_fps = 6  # The number of frames per second in the generated movie
initial_padding = 5  # The number of seconds to display the initial fluxes at the start of the video
final_padding = 5  # The number of seconds to display the final fluxes at the end of the video


species_path = None
java = False            # always False
settings = None
chemkin_output = ''     # this will be generated automatically
central_species_list = None
superimpose = False     # this will always be false, delete it
save_states = False
read_states = False     # fine to keep this always false and delete relevant code below
diffusion_limited = True
check_duplicates = True

## Load Mechanisms

In [ ]:
rmg_input_file = '/home/moon/autoscience/fuels/butane_20240403/simplified_input.py'  # for conditions <---- Note only one T,P,X in the reactor


diagram_base_name = 'my_mech'
os.makedirs(diagram_base_name, exist_ok=True)
mech_1_inp = '/home/moon/autoscience/aramco/truncated/aramco_truncated.inp'
mech_1_dict = '/home/moon/autoscience/aramco/species_dictionary.txt'
mech_1_label = 'aramco'
t1_match = 0.06535855787324196  # <--- tries to align the times given
mech_2_inp = '/home/moon/autoscience/fuels/butane_20240501/chem_annotated.inp'
mech_2_dict = '/home/moon/autoscience/fuels/butane_20240501/species_dictionary.txt'
mech_2_label = 'no lib 20240501'
t2_match = 0.02934108246263338



if species_path is None:
    species_path = os.path.join(os.path.dirname(rmg_input_file), 'species')
    generate_images = True
else:
    generate_images = False

print('Loading RMG job 1...')
rmg_job1 = rmgpy.tools.fluxdiagram.load_rmg_job(
    rmg_input_file,
    mech_1_inp,
    mech_1_dict,
    generate_images=generate_images,
    check_duplicates=check_duplicates
)

print('Loading RMG job 2...')
rmg_job2 = rmgpy.tools.fluxdiagram.load_rmg_job(
    rmg_input_file,
    mech_2_inp,
    mech_2_dict,
    generate_images=generate_images,
    check_duplicates=check_duplicates
)

Loading RMG job 1...


## Simulation

In [ ]:
print('Conducting simulation of reaction 1')
times1, concentrations1, reaction_rates1 = rmgpy.tools.fluxdiagram.simulate(
    rmg_job1.reaction_model,
    rmg_job1.reaction_systems[0],
    settings
)

print('Conducting simulation of reaction 2')
times2, concentrations2, reaction_rates2 = rmgpy.tools.fluxdiagram.simulate(
    rmg_job2.reaction_model,
    rmg_job2.reaction_systems[0],
    settings
)

In [ ]:
# Get the species and reactions corresponding to the provided concentrations and reaction rates
species_list1 = rmg_job1.reaction_model.core.species[:]
reaction_list1 = rmg_job1.reaction_model.core.reactions[:]
num_species1 = len(species_list1)

species_list2 = rmg_job2.reaction_model.core.species[:]
reaction_list2 = rmg_job2.reaction_model.core.reactions[:]
num_species2 = len(species_list2)

In [ ]:
# Compute the rates between each pair of species (big matrix warning!)
species_rates1 = np.zeros((len(times1), num_species1, num_species1), float)
for index1, reaction1 in enumerate(reaction_list1):
    rate1 = reaction_rates1[:, index1]
    if not reaction1.pairs: reaction1.generate_pairs()
    for reactant1, product1 in reaction1.pairs:
        reactant_index1 = species_list1.index(reactant1)
        product_index1 = species_list1.index(product1)
        species_rates1[:, reactant_index1, product_index1] += rate1
        species_rates1[:, product_index1, reactant_index1] -= rate1
        
species_rates2 = np.zeros((len(times2), num_species2, num_species2), float)
for index2, reaction2 in enumerate(reaction_list2):
    rate2 = reaction_rates2[:, index2]
    if not reaction2.pairs: reaction2.generate_pairs()
    for reactant2, product2 in reaction2.pairs:
        reactant_index2 = species_list2.index(reactant2)
        product_index2 = species_list2.index(product2)
        species_rates2[:, reactant_index2, product_index2] += rate2
        species_rates2[:, product_index2, reactant_index2] -= rate2

In [ ]:
# Determine the maximum concentration for each species and the maximum overall concentration
max_concentrations1 = np.max(np.abs(concentrations1), axis=0)
max_concentration1 = np.max(max_concentrations1)

# Determine the maximum reaction rates
max_reaction_rates1 = np.max(np.abs(reaction_rates1), axis=0)

# Determine the maximum rate for each species-species pair and the maximum overall species-species rate
max_species_rates1 = np.max(np.abs(species_rates1), axis=0)
max_species_rate1 = np.max(max_species_rates1)
species_index1 = max_species_rates1.reshape((num_species1 * num_species1)).argsort()


max_concentrations2 = np.max(np.abs(concentrations2), axis=0)
max_concentration2 = np.max(max_concentrations2)

# Determine the maximum reaction rates
max_reaction_rates2 = np.max(np.abs(reaction_rates2), axis=0)

# Determine the maximum rate for each species-species pair and the maximum overall species-species rate
max_species_rates2 = np.max(np.abs(species_rates2), axis=0)
max_species_rate2 = np.max(max_species_rates2)
species_index2 = max_species_rates2.reshape((num_species2 * num_species2)).argsort()


In [ ]:
# Determine the nodes and edges to keep
nodes1 = []
edges1 = []
for i in range(num_species1 * num_species1):
    product_index1, reactant_index1 = divmod(species_index1[-i - 1], num_species1)
    if reactant_index1 > product_index1:
        # Both reactant -> product and product -> reactant are in this list,
        # so only keep one of them
        continue
    if max_species_rates1[reactant_index1, product_index1] == 0:
        break
    if reactant_index1 not in nodes1 and len(nodes1) < max_node_count: nodes1.append(reactant_index1)
    if product_index1 not in nodes1 and len(nodes1) < max_node_count: nodes1.append(product_index1)
    if [reactant_index1, product_index1] not in edges1 and [product_index1, reactant_index1] not in edges1:
        edges1.append([reactant_index1, product_index1])
    if len(nodes1) > max_node_count:
        break
    if len(edges1) >= max_edge_count:
        break
        
nodes2 = []
edges2 = []
for i in range(num_species2 * num_species2):
    product_index2, reactant_index2 = divmod(species_index2[-i - 1], num_species2)
    if reactant_index2 > product_index2:
        # Both reactant -> product and product -> reactant are in this list,
        # so only keep one of them
        continue
    if max_species_rates2[reactant_index2, product_index2] == 0:
        break
    if reactant_index2 not in nodes2 and len(nodes2) < max_node_count: nodes2.append(reactant_index2)
    if product_index2 not in nodes2 and len(nodes2) < max_node_count: nodes2.append(product_index2)
    if [reactant_index2, product_index2] not in edges2 and [product_index2, reactant_index2] not in edges2:
        edges2.append([reactant_index2, product_index2])
    if len(nodes2) > max_node_count:
        break
    if len(edges2) >= max_edge_count:
        break


In [ ]:
species2_to_1 = {}
for i in range(len(species_list2)):
    for j in range(len(species_list1)):
        if species_list2[i].is_isomorphic(species_list1[j]):
            species2_to_1[i] = j
            break
    else:
        species2_to_1[i] = -1

In [ ]:
# edit nodes in 2 to match nodes in 1  and just thread them through to each other...
nodes2_edited = []
edges2_edited = []

for n2 in nodes2:
    # check if the species it refers to is in n1
    if species2_to_1[n2] > 0:
        nodes2_edited.append(species2_to_1[n2])
    else:
        nodes2_edited.append(n2)

for e2 in edges2:
    # check if the species it refers to is in n1
    new_edge = [e2[0], e2[1]]
    if species2_to_1[e2[0]] > 0:
        new_edge[0] = species2_to_1[e2[0]]
    if species2_to_1[e2[1]] > 0:
        new_edge[1] = species2_to_1[e2[1]]
    edges2_edited.append(new_edge)


In [ ]:
# Create the master graph all at once
my_red = '#ff000080'
my_blue = '#0000ff80'

t1s = np.arange(np.abs(times1 - t1_match).argmin() - 20, np.abs(times1 - t1_match).argmin() + 3)
t2s = np.arange(np.abs(times2 - t2_match).argmin() - 20, np.abs(times2 - t2_match).argmin() + 3)
# t1s = [len(times1) - 1]
# t2s = [len(times2) - 1]
for t in range(len(t1s)):
    slope = -max_node_pen_width / np.log10(concentration_tol)
    
    t1 = t1s[t]
    t2 = t2s[t]

    graph = pydot.Dot('flux_diagram', graph_type='digraph', overlap="false")
    graph.set_rankdir('LR')
    graph.set_fontname('sans')
    graph.set_fontsize('10')


    # Add a node for each species
    for index1 in nodes1:
        species1 = species_list1[index1]
        node1 = pydot.Node(name=str(species1))
        concentration1 = concentrations1[t1, index1] / max_concentration1
        if concentration1 < concentration_tol:
            penwidth = 0.0
        else:
            penwidth = round(slope * np.log10(concentration1) + max_node_pen_width, 3)
        node1.set_penwidth(penwidth)
        
        # Try to use an image instead of the label
        species_index1 = str(species1) + '.png'
        image_path1 = ''
        if not species_path or not os.path.exists(species_path):
            continue
        for root, dirs, files in os.walk(species_path):
            for f in files:
                if f == species_index1:
                    image_path1 = os.path.join(root, f)
                    break
        if os.path.exists(image_path1):
            node1.set_image(image_path1)
            node1.set_label(" ")
        graph.add_node(node1)

    for index2 in nodes2:
        if species2_to_1[index2] > 0:
            species2 = species_list1[species2_to_1[index2]]
            names = [x.get_name() for x in graph.get_node_list()]
            if str(species2) in names:
                continue  # node is already in graph
        else:
            species2 = species_list2[index2]
        node2 = pydot.Node(name=str(species2))  # check if it's already in there?? probably doesn't matter. We're overwriting
        concentration2 = concentrations2[t2, index2] / max_concentration2
        if concentration2 < concentration_tol:
            penwidth = 0.0
        else:
            penwidth = round(slope * np.log10(concentration2) + max_node_pen_width, 3)
        node2.set_penwidth(penwidth)
        
        # Try to use an image instead of the label
        species_index2 = str(species2) + '.png'
        image_path2 = ''
        if not species_path or not os.path.exists(species_path):
            continue
        for root, dirs, files in os.walk(species_path):
            for f in files:
                if f == species_index2:
                    image_path2 = os.path.join(root, f)
                    break
        if os.path.exists(image_path2):
            node2.set_image(image_path2)
            node2.set_label(" ")
        graph.add_node(node2)


    # Add an edge for each species-species rate
    slope = -max_edge_pen_width / np.log10(species_rate_tol)
    for reactant_index1, product_index1 in edges1:
        if reactant_index1 in nodes1 and product_index1 in nodes1:
            reactant1 = species_list1[reactant_index1]
            product1 = species_list1[product_index1]
            edge1 = pydot.Edge(str(reactant1), str(product1), color=my_red)
            species_rate1 = species_rates1[t1, reactant_index1, product_index1] / max_species_rate1
            if species_rate1 < 0:
                edge1.set_dir("back")
                species_rate1 = -species_rate1
            else:
                edge1.set_dir("forward")
            # Set the edge pen width
            if species_rate1 < species_rate_tol:
                penwidth = 0.0
                edge1.set_dir("none")
            else:
                penwidth = round(slope * np.log10(species_rate1) + max_edge_pen_width, 3)
            edge1.set_penwidth(penwidth)
            
            graph.add_edge(edge1)

    
    for reactant_index2, product_index2 in edges2:
        if reactant_index2 in nodes2 and product_index2 in nodes2:
            if species2_to_1[reactant_index2] > 0:
                reactant2 = species_list1[species2_to_1[reactant_index2]]
            else:
                reactant2 = species_list2[reactant_index2]
            
            if species2_to_1[product_index2] > 0:
                product2 = species_list1[species2_to_1[product_index2]]
            else:
                product2 = species_list2[product_index2]
            edge2 = pydot.Edge(str(reactant2), str(product2), color=my_blue)
            species_rate2 = species_rates2[t2, reactant_index2, product_index2] / max_species_rate2
            if species_rate2 < 0:
                edge2.set_dir("back")
                species_rate2 = -species_rate2
            else:
                edge2.set_dir("forward")
            # Set the edge pen width
            if species_rate2 < species_rate_tol:
                penwidth = 0.0
                edge2.set_dir("none")
            else:
                penwidth = round(slope * np.log10(species_rate2) + max_edge_pen_width, 3)
            edge2.set_penwidth(penwidth)
            graph.add_edge(edge2)
            
    # Generate the coordinates for all of the nodes using the specified program
    graph = pydot.graph_from_dot_data(graph.create_dot(prog=program).decode('utf-8'))[0]
    
    graph.add_node(pydot.Node(mech_1_label + f'\nt={times1[t1]:.4e}', label=mech_1_label + f'\nt={times1[t1]:.4e}', color=my_red, shape='box', penwidth=max_node_pen_width))
    graph.add_node(pydot.Node(mech_2_label + f'\nt={times2[t2]:.4e}', label=mech_2_label + f'\nt={times2[t2]:.4e}', color=my_blue, shape='box', penwidth=max_node_pen_width))
    
    
    graph.write_dot(os.path.join(diagram_base_name, f'{diagram_base_name}_{t1}.dot')) # Yes this is supposed to be an index, not an actual time
    graph.write_png(os.path.join(diagram_base_name, f'{diagram_base_name}_{t1}.png'))
#     break